In [1]:
import digitalhub as dh
import pandas as pd
import os

In [2]:
PROJECT = "demo-ml"
project = dh.get_or_create_project(PROJECT)

In [3]:
%%writefile data-prep.py

import pandas as pd
from sklearn.datasets import load_breast_cancer

from digitalhub_runtime_python import handler

@handler(outputs=["dataset"])
def breast_cancer_generator():
    """
    A function which generates the breast cancer dataset
    """
    breast_cancer = load_breast_cancer()
    breast_cancer_dataset = pd.DataFrame(
        data=breast_cancer.data, columns=breast_cancer.feature_names
    )
    breast_cancer_labels = pd.DataFrame(data=breast_cancer.target, columns=["target"])
    breast_cancer_dataset = pd.concat(
        [breast_cancer_dataset, breast_cancer_labels], axis=1
    )

    return breast_cancer_dataset

Writing data-prep.py


In [4]:
data_gen_fn = project.new_function(
                         name="data-prep",
                         kind="python",
                         python_version="PYTHON3_9",
                         source={"source": "data-prep.py", "handler": "breast_cancer_generator"})


In [5]:
gen_data_run = data_gen_fn.run(action="job", outputs={"dataset": "dataset"}, local_execution=True)

2024-08-26 11:02:56,622 - INFO - Validating task.
2024-08-26 11:02:56,623 - INFO - Validating run.
2024-08-26 11:02:56,623 - INFO - Starting task.
2024-08-26 11:02:56,624 - INFO - Configuring execution.
2024-08-26 11:02:56,949 - INFO - Composing function arguments.
2024-08-26 11:02:56,950 - INFO - Executing run.
2024-08-26 11:02:57,142 - INFO - Task completed, returning run status.


In [6]:
gen_data_run.outputs()["dataset"].as_df().head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [7]:
%%writefile train-model.py

import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

from digitalhub_runtime_python import handler
from sklearn.svm import SVC 
from pickle import dump
import sklearn.metrics
import os

@handler(outputs=["dataset"])
def train(project, di):

    df_cancer = di.as_df()
    X = df_cancer.drop(['target'],axis=1)
    y = df_cancer['target']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state=5)
    svc_model = SVC()
    svc_model.fit(X_train, y_train)
    y_predict = svc_model.predict(X_test)
    
    if not os.path.exists("model"):
        os.makedirs("model")
    
    with open("model/cancer_classifier.pkl", "wb") as f:
        dump(svc_model, f, protocol=5)

    metrics = {
        "f1_score": sklearn.metrics.f1_score(y_test, y_predict),
        "accuracy": sklearn.metrics.accuracy_score(y_test, y_predict),
        "precision": sklearn.metrics.precision_score(y_test, y_predict),
        "recall": sklearn.metrics.recall_score(y_test, y_predict),
    }
    project.log_model(
            name="cancer_classifier", 
            kind="sklearn", 
            source="./model/", 
            metrics=metrics
    )

Writing train-model.py


In [8]:
train_fn = project.new_function(
                         name="train",
                         kind="python",
                         python_version="PYTHON3_9",
                         source={"source": "train-model.py", "handler": "train"},
                         requirements=["scikit-learn==1.2.2"])

In [9]:
train_run = train_fn.run(action="job", inputs={"di": gen_data_run.outputs()["dataset"].key}, local_execution=False)

In [10]:
model = project.get_model("cancer_classifier")
model.spec.path

's3://datalake/demo-ml/model/cancer_classifier/28014f5d-7c2a-44ee-848a-0343b118d046/'

In [11]:
func = project.new_function(name="serve_sklearnmodel",
                            kind="sklearnserve",
                            path=model.spec.path)

serve_run = func.run(action="serve")

In [14]:
SERVICE_URL = serve_run.refresh().status.to_dict()["service"]["url"]

In [15]:
from sklearn.model_selection import train_test_split

df_cancer = gen_data_run.outputs()["dataset"].as_df()
X = df_cancer.drop(['target'],axis=1)
y = df_cancer['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state=5)
test_input = X_test.head(2).to_numpy().tolist()

In [16]:
from sklearn.model_selection import train_test_split

df_cancer = gen_data_run.outputs()["dataset"].as_df()
X = df_cancer.drop(['target'],axis=1)
y = df_cancer['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state=5)

import requests

with requests.post(f'http://{SERVICE_URL}/v2/models/model/infer', json={"inputs": [
    {
      "name": "input-0",
      "shape": [2, 30],
      "datatype": "FP32",
      "data": test_input
    }
  ]}) as r:
    res = r.json()
print(res)


{'model_name': 'model', 'model_version': None, 'id': 'd4890b36-3f62-485f-880f-021263a1c8c2', 'parameters': None, 'outputs': [{'name': 'output-0', 'shape': [2], 'datatype': 'INT64', 'parameters': None, 'data': [0, 1]}]}
